In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import time
%matplotlib notebook
import seaborn as sns
from scipy.special import expit as logit
from sklearn.model_selection import train_test_split
from scipy.special import expit as sigmoid # is more stable in case of overflows
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, \
recall_score, precision_score, accuracy_score, confusion_matrix
import math


# Prepare data

In [3]:
df_cleaned = pd.read_csv('/data/ortho/AllPatients.csv',  sep= ';')


In [4]:
# bereken symmetrie
df_cleaned['clavicula_x_dif'] = np.absolute(df_cleaned['clavicula_l_x'] - df_cleaned['clavicula_r_x'])
df_cleaned['clavicula_y_dif'] = np.absolute(df_cleaned['clavicula_l_y'] - df_cleaned['clavicula_r_y'])
df_cleaned['clavicula_z_dif'] = np.absolute(df_cleaned['clavicula_l_z'] - df_cleaned['clavicula_r_z'])

df_cleaned['scapula_x_dif'] = np.absolute(df_cleaned['scapula_l_x'] - df_cleaned['scapula_r_x'])
df_cleaned['scapula_y_dif'] = np.absolute(df_cleaned['scapula_l_y'] - df_cleaned['scapula_r_y'])
df_cleaned['scapula_z_dif'] = np.absolute(df_cleaned['scapula_l_z'] - df_cleaned['scapula_r_z'])

df_cleaned['humerus_x_dif'] = np.absolute(df_cleaned['humerus_l_x'] - df_cleaned['humerus_r_x'])
df_cleaned['humerus_y_dif'] = np.absolute(df_cleaned['humerus_l_y'] - df_cleaned['humerus_r_y'])
df_cleaned['humerus_z_dif'] = np.absolute(df_cleaned['humerus_l_z'] - df_cleaned['humerus_r_z'])

# hulp array, met alle parameters die voor de classifier gebruikt worden, je kan hier alles in doen wat je wilt
param = [ \
          'humerus_l_x', 'humerus_l_y', 'humerus_l_z', 'humerus_r_x', 'humerus_r_y', 'humerus_r_z', \
          'clavicula_l_x', 'clavicula_l_y', 'clavicula_l_z', 'clavicula_r_x', 'clavicula_r_y', 'clavicula_r_z', \
          'scapula_l_x', 'scapula_l_y', 'scapula_l_z', 'scapula_r_x', 'scapula_r_y', 'scapula_r_z', \
          'clavicula_x_dif','clavicula_y_dif','clavicula_z_dif', \
          'scapula_x_dif','scapula_y_dif','scapula_z_dif', \
          'humerus_x_dif', 'humerus_y_dif', 'humerus_z_dif'
         ]

df_cleaned['bias'] = 1

# split oorsprong kolom in onderdelen
x,y = df_cleaned['Oorsprong'].str.split(".").str #Oordprong word vertaald naar een string en wordt gesplits op de punt
df_cleaned['cat'],df_cleaned['pat'],df_cleaned['meting'],df_cleaned['oef'] = x.str.split("_").str #4 categorieen gemaakt obv file name
df_cleaned['cat'] = [ int(x[3:]) for x in df_cleaned['cat']] #voor elk 3+ element in de kolom wordt vertaald naar een int
df_cleaned['meting'] = [ int(x[6:]) for x in df_cleaned['meting']] 
df_cleaned['oef'] = [ int(x[3:]) for x in df_cleaned['oef']] 
df_cleaned['pat'] = [ int(x[3:]) for x in df_cleaned['pat']] 
#na deze regels te hebben uitgevoerd zijn er nieuwe categorieen met ints.

df_cleaned['pat'] = df_cleaned['cat']*1000+df_cleaned['pat'] #geef elke patient een uniek nummer

#maak boolean kolom per categorie
df_cleaned['c4'] = ['Cat4' in vincent for vincent in df_cleaned['Oorsprong']]
df_cleaned['c3'] = ['Cat3' in vincent for vincent in df_cleaned['Oorsprong']]
df_cleaned['c2'] = ['Cat2' in vincent for vincent in df_cleaned['Oorsprong']]
df_cleaned['c1'] = ['Cat1' in vincent for vincent in df_cleaned['Oorsprong']]

df_cleaned['y'] = [('Cat1' or 'Cat2') in vincent for vincent in df_cleaned['Oorsprong']]


# df_cleaned = df_cleaned[~df_cleaned.c3]
# df_cleaned = df_cleaned[~df_cleaned.c2]



Xcolumns = ['bias']
Xcolumns.extend(param)

X = df_cleaned[Xcolumns]
y = df_cleaned['y']


In [4]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

lr = LogisticRegression()
lr.fit(X_train, y_train)

y_true = y_test
y_pred = lr.predict(X_test)

TN, FP, FN, TP = confusion_matrix(y_true, y_pred).ravel()

print(lr.coef_[0])

tab = [["pred pos", TP, FP], ["pred neg", FN, TN]]
print(pd.DataFrame(tab, columns=["", "pos", "neg"]))
print()
print("recall: ", recall_score(y_true, y_pred))
print("precision: ", precision_score(y_true, y_pred))
print("accuracy: ", accuracy_score(y_true, y_pred.round().astype(bool)))
print('\n\n')




/opt/jupyterhub/anaconda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


[-2.15283181  0.02505178  0.03126563  0.02572596 -0.01942919  0.00475239
 -0.03069388 -0.09587018 -0.07563774 -0.04825915  0.04745688  0.1127199
  0.06812448 -0.0447838  -0.10926716  0.01746892  0.04860239 -0.08211895
 -0.03583993 -0.03594507 -0.07128507 -0.11013998  0.01022334  0.05383951
  0.01797442 -0.00705636 -0.04644432 -0.01875265]
              pos    neg
0  pred pos  3693   1433
1  pred neg  3039  26124

recall:  0.5485739750445633
precision:  0.7204447912602419
accuracy:  0.8695791653299892





In [5]:
print(y_pred)

[False False False ... False False False]


In [6]:
y_pred.round().astype(bool)

array([False, False, False, ..., False, False, False])